In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

: 

In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=aZ7lpA7aYOA9NdF2CHR0azNb1P7J5h&access_type=offline&code_challenge=UCck6AWxyr_o7V_qMnq0B5aS7s3uZlnqs4Wq0oRvgsg&code_challenge_method=S256


You are now logged in as [ugyung87@gmail.com].
Your current project is [sesac-24-123].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
# vertex AI STUDIO 방식 
# GCP로 STT/TTS/Gemini 

# 아래 셀이 정상 동작 시, 로그인 문제없고, 파이썬 패키지 설치 문제없고, gcp에 원활하게 엑세스 된 것 

In [2]:
PROJECT_ID = "sesac-24-123"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
from IPython.display import Markdown

In [4]:
model = GenerativeModel("gemini-1.5-flash")

In [5]:
prompt = "한국의 수도는 어디야? ?"

In [6]:
# 마크다운까지 작동하게 함 
display(Markdown(model.generate_content(prompt).text))

 한국의 수도는 **서울**입니다. 


In [7]:
print(model.generate_content(prompt).text)

 한국의 수도는 **서울**입니다. 



In [13]:
from google.cloud import speech
import pyaudio
import keyboard  # keyboard 라이브러리 임포트
import os

# Google Cloud Speech-to-Text 클라이언트 초기화
client = speech.SpeechClient()

# PyAudio 초기화
p = pyaudio.PyAudio()

# 마이크 입력 설정
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=48000,
                input=True,
                frames_per_buffer=1024)

# 음성 데이터를 담을 버퍼
audio_content = []

print("녹음 시작 (말하기 시작하세요). 'q' 키를 누르면 녹음이 종료됩니다.")

# 음성 녹음 루프
while True:
    # 마이크에서 데이터 읽기
    data = stream.read(1024)
    audio_content.append(data)

    # 'q' 키를 누르면 녹음 종료
    if keyboard.is_pressed('q'):  # keyboard 라이브러리 사용
        print("녹음 종료")
        break

# 스트림 닫기
stream.stop_stream()
stream.close()

# PyAudio 닫기
p.terminate()

# 음성 데이터를 합치기qq
audio_content = b''.join(audio_content)

# 음성 인식 설정
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=48000,
    language_code="ko-KR",  # 한국어로 설정
    model="default",
    audio_channel_count=1,
    enable_word_confidence=True,
    enable_word_time_offsets=True,
)

# 음성 데이터를 Speech-to-Text API로 전송
audio = speech.RecognitionAudio(content=audio_content)
operation = client.long_running_recognize(config=config, audio=audio)

print("음성 인식 진행 중...")

# 인식 결과를 기다림
response = operation.result(timeout=90)

# 파일 이름에 숫자를 붙이기
base_filename = "transcript"
file_number = 1
while os.path.exists(f"{base_filename}{file_number}.txt"):
    file_number += 1
filename = f"{base_filename}{file_number}.txt"

# 결과를 새 파일에 저장
with open(filename, "w", encoding="utf-8") as f:
    for result in response.results:
        transcript = result.alternatives[0].transcript
        f.write(f"Transcript: {transcript}\n")
        # print(f"Transcript: {transcript}")
        print(model.generate_content(result.alternatives[0].transcript).text)        

print("음성 인식 결과가 transcript.txt 파일에 저장되었습니다.")



녹음 시작 (말하기 시작하세요). 'q' 키를 누르면 녹음이 종료됩니다.
녹음 종료
음성 인식 진행 중...
무엇을 따로 할까요? 더 자세한 정보를 주세요! 예를 들어, "따로 뭘 해야 할지 모르겠어. 뭘 해야 할지 추천해줄 수 있어?" 또는 "따로 뭘 먹을지 고민이야. 맛있는 거 추천해줄 수 있어?" 와 같이 말씀해주세요. 

음성 인식 결과가 transcript.txt 파일에 저장되었습니다.


In [18]:
import re

def markdown_to_text(markdown_text):
  """
  마크다운 텍스트를 일반 텍스트로 변환하는 함수입니다.
  Args:
    markdown_text: 변환할 마크다운 텍스트
  Returns:
    일반 텍스트
  """

  # 1. 헤더 제거
  text = re.sub(r'\#+ ', '', markdown_text)
  # 2. 굵은 글씨 제거
  text = re.sub(r'\\([^]+)\\*', r'\1', text)
  # 3. 기타 마크다운 태그 제거
  text = re.sub(r'<[^>]+>', '', text)
  # 4. 줄 바꿈 제거
  text = text.replace('\n', ' ')
  return text.strip()

# 예시:
markdown_text = "한국의 수도는 **서울**입니다."
plain_text = markdown_to_text(markdown_text)
print(plain_text)

error: unterminated character set at position 3

In [20]:
import re

def markdown_to_text(markdown_text):
    """
    마크다운 텍스트를 일반 텍스트로 변환하는 함수입니다.
    Args:
        markdown_text: 변환할 마크다운 텍스트
    Returns:
        일반 텍스트
    """

    # 1. 헤더 제거
    text = re.sub(r'#* ', '', markdown_text)
    
    # 2. 굵은 글씨 및 기울임 글씨 제거
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)  # **bold**
    text = re.sub(r'\*(.*?)\*', r'\1', text)      # *italic*
    text = re.sub(r'__(.*?)__', r'\1', text)      # __bold__
    text = re.sub(r'_(.*?)_', r'\1', text)        # _italic_

    # 3. 링크 제거 (텍스트 부분만 남기기)
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)

    # 4. HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 5. 리스트, 줄 바꿈 등을 처리하여 불필요한 공백 제거
    # text = re.sub(r'\n+', ' ', text)  # 여러 줄 바꿈을 하나로
    # text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나로

    return text.strip()

# 예시:
# markdown_text = "한국의 수도는 **서울**입니다."
markdown_text = transcript
plain_text = markdown_to_text(markdown_text)
print(plain_text)

뭐라고따조


In [8]:
# 녹음파일 만들기 
# pip install google-cloud-texttospeech

"""Synthesizes speech from the input string of text."""
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()

input_text = texttospeech.SynthesisInput(text=transcript)

# Note: the voice can also be specified by name.
# Names of voices can be retrieved with client.list_voices().
voice = texttospeech.VoiceSelectionParams(
    # language_code="en-US",
    # name="en-US-Studio-O",
    language_code="ko-KR",  # 한국어로 변경
    name="ko-KR-Standard-A",  # 한국어 음성 이름 선택
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.LINEAR16,
    speaking_rate=1     # 말하는 속도 
)

response = client.synthesize_speech(
    request={"input": input_text, "voice": voice, "audio_config": audio_config}
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

NameError: name 'transcript' is not defined